# 5. LeNet-5 Model Evaluation on Test Data

After training our LeNet-5 model, the next crucial step is to evaluate its performance on the unseen test set. This gives us an unbiased measure of how well our model generalizes to new data. We will apply the model, store its predictions in FiftyOne, and then use FiftyOne's powerful evaluation tools to analyze the results in detail.

**Key concepts covered:**
*   Applying a PyTorch model to a FiftyOne dataset
*   Storing predictions, confidence, and logits
*   Evaluating classification performance
*   Analyzing prediction confidence distributions
*   Computing sample hardness and mistakenness

![](https://github.com/andandandand/fiftyone/blob/develop/docs/source/getting_started_experiences/Classification/assets/mistakenness_test_set_decreasing.png?raw=1)

## Setup

We need to reload our datasets and redefine our model architecture and helper classes to apply the trained model.

In [17]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as Fun
import torchvision.transforms.v2 as transforms
from torch.utils.data import Dataset

import fiftyone as fo
import fiftyone.brain as fob
from fiftyone import ViewField as F

# Redefine the model architecture so we can load the weights
class ModernLeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(ModernLeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=4)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(Fun.relu(self.conv1(x)))
        x = self.pool(Fun.relu(self.conv2(x)))
        x = Fun.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = Fun.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Redefine the custom dataset class
class CustomTorchImageDataset(torch.utils.data.Dataset):
    def __init__(self, fiftyone_dataset, image_transforms=None, label_map=None, gt_field="ground_truth"):
        self.fiftyone_dataset = fiftyone_dataset
        self.image_paths = self.fiftyone_dataset.values("filepath")
        self.str_labels = self.fiftyone_dataset.values(f"{gt_field}.label")
        self.image_transforms = image_transforms
        self.label_map = label_map if label_map is not None else {str(i): i for i in range(10)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('L')
        if self.image_transforms: image = self.image_transforms(image)
        label_str = self.str_labels[idx]
        label_idx = self.label_map.get(label_str, -1)
        return image, torch.tensor(label_idx, dtype=torch.long)

### Reload the Best LeNet Model

We load the saved model weights that achieved the best validation performance during training. This ensures we are evaluating the most generalizable version of our model.

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_save_path = Path(os.getcwd()) / 'best_lenet.pth'

loaded_model = ModernLeNet5().to(device)
loaded_model.load_state_dict(torch.load(model_save_path, map_location=device))
loaded_model.eval()

print(f"Model loaded successfully from {model_save_path}")

Model loaded successfully from /root/workspace/fiftyone/docs/source/getting_started_experiences/Classification/best_lenet.pth


## Apply the Model to the Test Set

We'll now run inference on the entire test set. We'll collect the predictions, confidence scores, and raw logits for each sample and store them back into our FiftyOne dataset. Storing predictions as structured `fo.Classification` objects allows for rich and interactive analysis, such as retrieving the samples where the labeling confidence is within querying thresholds.

In [19]:
# Load datasets
test_dataset = fo.load_dataset("mnist-test-set")
train_dataset = fo.load_dataset("mnist-training-set")

# Recreate transforms using stats from the training set
mean_intensity, std_intensity = 0.1307, 0.3081 # Pre-computed for simplicity
image_transforms = transforms.Compose([
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize((mean_intensity,), (std_intensity,))
])
dataset_classes = sorted(test_dataset.distinct("ground_truth.label"))
label_map = {label: i for i, label in enumerate(dataset_classes)}

# Create DataLoader for the test set
torch_test_set = CustomTorchImageDataset(test_dataset, image_transforms=image_transforms, label_map=label_map)
test_loader = torch.utils.data.DataLoader(torch_test_set, batch_size=64, num_workers=os.cpu_count())

# Run inference
predictions, all_logits = [], []
with torch.inference_mode():
    for images, _ in tqdm(test_loader):
        images = images.to(device)
        logits = loaded_model(images)
        all_logits.append(logits.cpu().numpy())
        _, predicted = torch.max(logits.data, 1)
        predictions.extend(predicted.cpu().numpy())

all_logits = np.concatenate(all_logits, axis=0)

# Store predictions back in FiftyOne
with fo.ProgressBar(total=len(test_dataset)) as pb:
    for i, sample in enumerate(test_dataset):
        pred_idx = predictions[i]
        sample_logits = all_logits[i]
        conf = float(Fun.softmax(torch.tensor(sample_logits), dim=0).numpy()[pred_idx])
        sample["lenet_classification"] = fo.Classification(
            label=dataset_classes[pred_idx],
            confidence=conf,
            logits=sample_logits.tolist()
        )
        sample.save()
        pb.update()

  0%|          | 0/157 [00:00<?, ?it/s]

100%|██████████| 157/157 [00:04<00:00, 33.71it/s] 

   0% |/------------|    23/10000 [112.9ms elapsed, 49.0s remaining, 203.7 samples/s] 

 100% |█████████████| 10000/10000 [23.0s elapsed, 0s remaining, 438.1 samples/s]      


INFO	eta.core.utils:utils.py:_draw()-  100% |█████████████| 10000/10000 [23.0s elapsed, 0s remaining, 438.1 samples/s]      


## Evaluating LeNet's Performance

With predictions stored, we can use `evaluate_classifications()` again to get a performance report and confusion matrix for our custom LeNet model. We expect a significant improvement over CLIP's zero-shot performance.

In [21]:
session = fo.launch_app(test_dataset, auto=False)
lenet_evaluation_results = test_dataset.evaluate_classifications(
    "lenet_classification",
    gt_field="ground_truth",
    eval_key="lenet_eval")

print(session.url)

Session launched. Run `session.show()` to open the App in a cell output.


INFO	fiftyone.core.session.session:session.py:launch_app()- Session launched. Run `session.show()` to open the App in a cell output.


http://0.0.0.0:5151/


In [13]:
lenet_evaluation_results.print_report(digits=3)


              precision    recall  f1-score   support

    0 - zero      0.996     0.993     0.994       980
     1 - one      0.998     0.990     0.994      1135
     2 - two      0.988     0.993     0.991      1032
   3 - three      0.990     0.988     0.989      1010
    4 - four      0.983     0.993     0.988       982
    5 - five      0.978     0.990     0.984       892
     6 - six      0.997     0.987     0.992       958
   7 - seven      0.980     0.991     0.985      1028
   8 - eight      0.988     0.989     0.988       974
    9 - nine      0.991     0.975     0.983      1009

    accuracy                          0.989     10000
   macro avg      0.989     0.989     0.989     10000
weighted avg      0.989     0.989     0.989     10000



The accuracy should be >99%, a huge leap from CLIP's ~28%. The confusion matrix is also much cleaner, with most values concentrated on the diagonal.

In [14]:
lenet_evaluation_results.plot_confusion_matrix()

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',
              'uid': 'd5774b03-5e0a-4391-a4c4-0db9b1518576',
              'x': {'bdata': ('AAECAwQFBgcICQABAgMEBQYHCAkAAQ' ... 'kAAQIDBAUGBwgJAAECAwQFBgcICQ=='),
                    'dtype': 'i1'},
              'y': {'bdata': ('AAAAAAAAAAAAAAEBAQEBAQEBAQECAg' ... 'cICAgICAgICAgICQkJCQkJCQkJCQ=='),
                    'dtype': 'i1'}},
             {'colorscale': [[0.0, 'rgb(255,245,235)'], [0.125,
                             'rgb(254,230,206)'], [0.25, 'rgb(253,208,162)'],
                             [0.375, 'rgb(253,174,107)'], [0.5, 'rgb(253,141,60)'],
                             [0.625, 'rgb(241,105,19)'], [0.75, 'rgb(217,72,1)'],
                             [0.875, 'rgb(166,54,3)'], [1.0, 'rgb(127,39,4)']],
              'hoverinfo': 'skip',
              'showscale': False,
              'type': 'heatmap',
              'uid': '22ffdccc-e23f-49cb-92b5-5b7e4a1bee73',


## Comparing Model Predictions Side by Side within the FiftyOne App with the Model Evaluation Panel

We can go to the Model Evaluation Panel to gain better insights about how the fine-tuned LeNet performs vs CLIP's zero-shot predictions. We go to `+` -> `Model Evaluation` (next to `Samples`) to launch the panel. 

![](https://github.com/andandandand/fiftyone/blob/develop/docs/source/getting_started_experiences/Classification/assets/launch_model_eval_panel.webp?raw=1)

## Hardness and Mistakenness

Beyond accuracy, we can analyze the model's logits to understand sample-level difficulties.

- **Hardness**: Measures the model's prediction uncertainty. High hardness indicates samples the model found difficult, which are often edge cases.
- **Mistakenness**: Identifies samples where the model was confident but wrong. High mistakenness can often point to labeling errors in the dataset.

We use `fiftyone.brain` to compute these values.

In [15]:
fob.compute_hardness(test_dataset, label_field='lenet_classification')

fob.compute_mistakenness(test_dataset, 
                         pred_field="lenet_classification",
                         label_field="ground_truth")

session.refresh()
print("Hardness and mistakenness computed.")

Computing hardness...


INFO	fiftyone.brain.internal.core.hardness:hardness.py:compute_hardness()- Computing hardness...


   1% |-------------|    84/10000 [205.5ms elapsed, 24.3s remaining, 408.8 samples/s] 

 100% |█████████████| 10000/10000 [14.7s elapsed, 0s remaining, 723.3 samples/s]      


INFO	eta.core.utils:utils.py:_draw()-  100% |█████████████| 10000/10000 [14.7s elapsed, 0s remaining, 723.3 samples/s]      


...

 100% |█████████████| 10000/10000 [12.6s elapsed, 0s remaining, 762.6 samples/s]      


INFO	eta.core.utils:utils.py:_draw()-  100% |█████████████| 10000/10000 [12.6s elapsed, 0s remaining, 762.6 samples/s]      


Mistakenness computation complete


INFO	fiftyone.brain.internal.core.mistakenness:mistakenness.py:compute_mistakenness()- Mistakenness computation complete


Hardness and mistakenness computed.


Let's view the samples with the highest mistakenness scores. These are the prime candidates for being mislabeled.

In [16]:
mistakenness_quantiles = test_dataset.quantiles("mistakenness", [0.99])

suspicious_test_samples_view = test_dataset.match(
                             F("mistakenness") > mistakenness_quantiles[-1]
                             ).sort_by("mistakenness", reverse=True)

session.view = suspicious_test_samples_view
print(f"Displaying {len(suspicious_test_samples_view)} most mistaken samples in the App: {session.url}")

Displaying 100 most mistaken samples in the App: http://0.0.0.0:5151/


## Exercises

1. Create and save views of samples with high `hardness` and filter them further by `ground_truth.label` and `lenet_classification_label`
2. Repeat the process in 1. for `mistakenness`.  Which digits do you find most quirky? What impact do these have on the evaluation of the model?  

![](https://github.com/andandandand/fiftyone/blob/develop/docs/source/getting_started_experiences/Classification/assets/weird_test_digit.webp?raw=1)

## Next Steps

We've thoroughly evaluated our LeNet model on the test set. But what did the model actually *learn*? 

In the next notebook, we'll dive into the model's internal representations by extracting embeddings from its hidden layers on the *training data*. This will help us understand the features it learned and analyze the training set for quality issues.

Proceed to `6_lenet_feature_analysis.ipynb`.